In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report
import pickle
with open("/kaggle/input/input-topics/df_topics.pkl", "rb") as f:
    df_topics = pickle.load(f)

# Sliding_Window

In [3]:
# Tách X, y từ df_topics
X = df_topics.drop(columns=["label"]).values  # Chuyển thành numpy array
y = df_topics["label"].values

In [ ]:
# Chia tập train/test trước KHI scale và tạo cửa sổ
train_size = int(len(X) * 0.8)
X_train_raw, X_test_raw = X[:train_size], X[train_size:]
y_train_raw, y_test_raw = y[:train_size], y[train_size:]

# Scale dữ liệu RIÊNG BIỆT trên từng tập
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_raw)
X_test_scaled = scaler.transform(X_test_raw)

# Hàm tạo cửa sổ trượt (chỉ dùng features để tạo X_window)
def create_sliding_windows(features, labels, window_size):
    X, y = [], []
    for i in range(len(features) - window_size):
        X.append(features[i:i + window_size])
        y.append(labels[i + window_size])  # Nhãn từ biến y
    return np.array(X), np.array(y)

# Tạo cửa sổ cho train và test
window_size = 100
X_train, y_train = create_sliding_windows(X_train_scaled, y_train_raw, window_size)
X_test, y_test = create_sliding_windows(X_test_scaled, y_test_raw, window_size)

# Danh sách các mô hình với RBF kernel, thay đổi tham số C và gamma
svm_models = {
    "rbf_C1_gamma_auto": SVC(kernel='rbf', C=1.0, gamma='auto', random_state=42),
    "rbf_C10_gamma_auto": SVC(kernel='rbf', C=10.0, gamma='auto', random_state=42),
    "rbf_C100_gamma_auto": SVC(kernel='rbf', C=100.0, gamma='auto', random_state=42),
    "rbf_C10_gamma_scale": SVC(kernel='rbf', C=10.0, gamma='scale', random_state=42),
    "rbf_C100_gamma_scale": SVC(kernel='rbf', C=100.0, gamma='scale', random_state=42)
}

# Đánh giá các mô hình
results = []
for name, model in svm_models.items():
    print(f"Evaluating model: {name}")
    model.fit(X_train.reshape(X_train.shape[0], -1), y_train)  # Chuyển đổi về dạng phù hợp
    y_pred = model.predict(X_test.reshape(X_test.shape[0], -1))
    
    accuracy = accuracy_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append((name, accuracy, mse, rmse, mae, r2))
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R2 Score: {r2:.4f}")
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("-" * 50)

# Hiển thị kết quả dưới dạng bảng
results_df = pd.DataFrame(results, columns=["Model", "Accuracy", "MSE", "RMSE", "MAE", "R2 Score"])
print(results_df.sort_values(by="Accuracy", ascending=False))


Evaluating model: rbf_C1_gamma_auto
